In [1]:
%config IPCompleter.greedy=True
import re
import json
from bs4 import BeautifulSoup
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import time

import base64
import xmltodict

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [8]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])

In [9]:
documents_by_id = {}
es.indices.delete(index='myandex')
es.indices.create(index='myandex')

{'acknowledged': True, 'index': 'myandex', 'shards_acknowledged': True}

In [10]:
def processFile(i):
    prefix = '../byweb_for_course/byweb.'
    suffix = '.xml'
    filename = prefix + str(i) + suffix
    with open(filename, 'rb') as f:
        decoded = f.read().decode('cp1251')
        xmldict = xmltodict.parse(decoded)
        for doc in tqdm(xmldict['romip:dataset']['document']):
            try:
                docID = doc['docID']
                documents_by_id[docID] = {}
                url = base64.b64decode(doc['docURL']).decode('cp1251')
                content = base64.b64decode(doc['content']['#text']).decode('cp1251')
                documents_by_id[docID]['url'] = url
                documents_by_id[docID]['content'] = content
            except Exception as e:
                print(e)

In [11]:
import pickle

def readTitles(i):
    with open('../../hw_1/docs' + str(i), 'rb') as f:
        for _ in range(20000):
            (docID, title) = pickle.load(f)
            documents_by_id[docID]['title'] = title

In [12]:
for i in range(10):
    processFile(i)
    readTitles(i)

In [13]:
settings_final = {
    'mappings': {
        'properties': {
            'url': {
                'type': 'text'
            },
            'content': {
                'type': 'text',
                "analyzer": "my_custom_analyzer"
            },
            'title': {
                'type': 'text',
                "analyzer": "my_custom_analyzer"
            }
        }
    },
    "settings": {
    "analysis": {
      "analyzer": {
        "my_custom_analyzer": {
          "type":      "custom", 
          "tokenizer": "standard",
          "char_filter": [
            "html_strip",
            "yont"
          ],
          "filter": [
            "lowercase",
           # "asciifolding",
            "russian_snow",
            "english_snow"
          ]
        }
      },
        'char_filter': {
                'yont': {
                    'type': 'mapping',
                    'mappings': [
                        'ё => е',
                        'Ё => Е'
                    ]
                }
            },
    'filter': {
            'stop_words': {
                'type': 'stop',
                'stopwords': [
                ]
            },
            'russian_snow': {
                'type': 'snowball',
                'language': 'russian'
            },
            'english_snow': {
                'type': 'snowball',
                'language': 'english'
            }
     }
    }
  }
}

In [14]:
def recreate_index():
    es.indices.delete(index='myandex')
    es.indices.create(index='myandex', body=settings_final)

In [15]:
recreate_index()

In [16]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='myandex', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [17]:
analyzer = {
    'analyzer': 'my_custom_analyzer'
}

check_analyzer(analyzer, '<meta http-equiv="Content-Type" content="text/html; charset=windows-1251"> bla bla русский countable текст Ёшкин кот')

['bla', 'bla', 'русск', 'countabl', 'текст', 'ешкин', 'кот']

In [18]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [19]:
def es_actions_generator():
    for doc_id, doc in tqdm(documents_by_id.items()):
        yield create_es_action('myandex', doc_id, doc)

In [20]:
start = time.time()
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)
end = time.time()
print(f"Time on index creation: {time.strftime('%H:%M:%S.%l', time.gmtime(end - start))}")
print(f"In seconds: {end - start}")


Time on index creation: 00:03:29.12
In seconds: 209.3906512260437


In [21]:
def search(query, *args):
    pretty_print_result(es.search(index='myandex', body=query, size=20), args)
    # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20

def raw_search(query):
    search_result = es.search(index='myandex', body=query, size=20)['hits']
    return [(hit['_id'], hit['_score']) for hit in search_result['hits']]
    
def pretty_print_result(search_result, fields=[]):
    # fields is a list of fields names which we want to be printed
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits'][:6]:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='myandex', id=doc_id)['_source']

In [22]:
def get_query(query):
    return {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'title': {
                            'query': query,
                            'boost': 0.2
                        }
                    }
                },
                {
                    'match': {
                        'content': query
                    }
                }
            ]
       }
    }
    }

q = get_query('<meta http-equiv="Content-Type" content="text/html; charset=windows-1251">')
search(q)
raw_search(q)

Total documents: 0


[]

In [23]:
def print_index_size(index): 
    print(f"Size of index: {es.indices.stats(index)['_all']['primaries']['store']['size_in_bytes'] / 2 ** 30} GB")

In [24]:
print_index_size('myandex')

Size of index: 3.7911960938945413 GB


In [25]:
def load_queries_and_relevance():
    relevance = defaultdict(dict)
    filename = '../relevant_table_2009.xml'
    with open(filename, 'rb') as f:
        xmldict = xmltodict.parse(f.read())
        for task in tqdm(xmldict['taskDocumentMatrix']['task']):
            task_rel = {}
            has_vital = False
            for doc in task['document']:
                if doc['@relevance'] == 'vital':
                    has_vital = True
                task_rel[doc['@id']] = doc['@relevance']
            if has_vital:
                relevance[task['@id']] = task_rel
    filename = '../web2008_adhoc.xml'
    with open(filename, 'rb') as f:
        xmldict = xmltodict.parse(f.read())
        for task in tqdm(xmldict['task-set']['task']):
            if task['@id'] in relevance:
                relevance[task['@id']]['querytext'] = task['querytext']
    return relevance

In [26]:
relevance = load_queries_and_relevance()

In [27]:
def get_number_of_correct_out_of_k(results, task_relevance, k):
    return sum([1 if res[0] in task_relevance and task_relevance[res[0]] == 'vital' else 0 for res in results[:k]])

def measure_performance():    
    Q = len(relevance)
    pq = 0
    rq = 0
    prq = 0
    mapq = 0
    for task in relevance.keys():
        sk = 0
        task_relevance = relevance[task]
        results = raw_search(get_query(task_relevance['querytext']))
        #if len(results) < 20:
        #    print("WARNING LESS 20")
        sk = get_number_of_correct_out_of_k(results, task_relevance, 20)
        pq += sk / 20
        relevant_size = len(['vital' for value in task_relevance.values() if value == 'vital'])
        rq += sk / relevant_size
        prq += get_number_of_correct_out_of_k(results, task_relevance, relevant_size) / relevant_size
        mapk = 0
        for k in range(1, 21):
            mapk += get_number_of_correct_out_of_k(results, task_relevance, k) / k
        mapk /= 20
        mapq += mapk
    print(f"p@20: {pq / Q}")
    print(f"r@20: {rq / Q}")
    print(f"p@R(q): {prq / Q}")
    print(f"map@20(q): {mapq / Q}")

In [28]:
measure_performance()

p@20: 0.39252525252525283
r@20: 0.2543251936253419
p@R(q): 0.21653136551511468
map@20(q): 0.4478636168079246


In [29]:
def get_empty_query():
    return {
    'query': {
        'bool': {
            'should': {
                'match': {
                    'content': ''
                }
            }
        }
    }
    }

q = get_empty_query()
search(q)
raw_search(q)

Total documents: 0


[]